# Assignment #3
## P556: Applied Machine Learning

More often than not, we will use a deep learning library (Tensorflow, Pytorch, or the wrapper known as Keras) to implement our models. However, the abstraction afforded by those libraries can make it hard to troubleshoot issues if we don't understand what is going on under the hood. In this assignment you will implement a fully-connected and a convolutional neural network from scratch. To simplify the implementation, we are asking you to implement static architectures, but you are free to support variable number of layers/neurons/activations/optimizers/etc. We recommend that you make use of private methods so you can easily troubleshoot small parts of your model as you develop them, instead of trying to figure out which parts are not working correctly after implementing everything. Also, keep in mind that there is code from your fully-connected neural network that can be re-used on the CNN. 

Problem #1.1 (40 points): Implement a fully-connected neural network from scratch. The neural network will have the following architecture:

- Input layer
- Dense hidden layer with 512 neurons, using relu as the activation function
- Dropout with a value of 0.2
- Dense hidden layer with 512 neurons, using relu as the activation function
- Dropout with a value of 0.2
- Output layer, using softmax as the activation function

The model will use categorical crossentropy as its loss function. 
We will optimize the gradient descent using RMSProp, with a learning rate of 0.001 and a rho value of 0.9.
We will evaluate the model using accuracy.

Why this architecture? We are trying to reproduce from scratch the following [example from the Keras documentation](https://keras.io/examples/mnist_mlp/). This means that you can compare your results by running the Keras code provided above to see if you are on the right track.

In [0]:
#import libraries
import numpy as np
import pandas as pd
import keras
import math
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [0]:
class NeuralNetwork(object):
  def __init__(self, epochs, learning_rate):
    self.epochs=epochs
    self.learning_rate=learning_rate
    self.weights_1=[]
    self.weights_2=[]
    self.weights_3=[]
    self.bias_1=[]
    self.bias_2=[]
    self.bias_3=[]
    self.grad_cache_1=[]
    self.grad_cache_2=[]
    self.grad_cache_3=[]
    #self.keep_prob_1=1-float(input('Enter dropout for layer 2'))
    #self.keep_prob_2=1-float(input('Enter dropout for layer 3'))
    #self.h_layer_1=int(input('Enter the number of neurons you want in the first hidden layer'))
    #self.h_layer_2=int(input('Enter the number of neurons you want in the second hidden layer'))
    self.keep_prob_1=1
    self.keep_prob_2=1
    self.h_layer_1=512
    self.h_layer_2=512
    self.db1_cache=[]
    self.db2_cache=[]
    self.db3_cache=[]


  
  def forward_prop(self,x_train):
    #First hidden layer
    z1=self.weights_1.dot(x_train)+self.bias_1
    a1=np.maximum(0,z1)
    self.d1=np.random.rand(a1.shape[0],a1.shape[1])<self.keep_prob_1  #Dropout matrix
    a1=np.multiply(a1,self.d1)
    a1/=self.keep_prob_1

    #Second hidden layer
    z2=self.weights_2.dot(a1)+self.bias_2
    a2=np.maximum(0,z2)
    self.d2=np.random.rand(a2.shape[0],a2.shape[1])<self.keep_prob_2   #Dropout matrix
    a2=np.multiply(a2,self.d2)
    a2/=self.keep_prob_2
    

    #Output layer
    z3=self.weights_3.dot(a2)+self.bias_3
    exp_z3=np.exp(z3)
    #print(exp_z3[0:5,0:5])
    for i in range(len(np.transpose(exp_z3))):
      maximum=max(np.transpose(exp_z3)[i])
      np.transpose(exp_z3)[i]/=maximum
    
    #print(exp_z3[0:5,0:5])
    for i in range(len(np.transpose(exp_z3))):
      exp_sum=sum(np.transpose(exp_z3)[i])
      np.transpose(exp_z3)[i]/=exp_sum
      #maximum=max(np.transpose(exp_z3)[i])

    #for i in np.transpose(exp_z3):
    #  print(sum(i))

    a3=exp_z3
    #print(a3[:,0])
    #print(np.sum(a3[:,0]))
    return [a1,a2,a3]
      
  def back_prop(self,x_train,y_train,a1,a2,a3):
    
    m=x_train.shape[1]
    
    #Output Layer
    print('a3-'+str(a3[:5,:5]))
    print('y_train-'+str(y_train[:5,:5]))
    dz3=a3-y_train
    print('dz3-'+str(dz3[:5,:5]))
    dw3=(1/m)*dz3.dot(np.transpose(a2))
    print('dw3-'+str(dw3[:5,:5]))
    db3=(1/m)*np.sum(dz3,axis=1,keepdims=True)
    print('db3- '+str(db3[:5,:5]))
    #2nd Hidden Layer
    print('a2- '+str(a2[:5,:5]))
    dz2=np.multiply(np.transpose(self.weights_3).dot(dz3),np.maximum(0,np.sign(a2)))
    print('dz2-'+str(dz2[:5,:5]))
    dw2=(1/m)*dz2.dot(np.transpose(a1))
    print('dw2-'+str(dw2[:5,:5]))
    db2=(1/m)*np.sum(dz2,axis=1,keepdims=True)
    print('db2- '+str(db2[:5,:5]))

    #1st Hidden Layer
    print('a1- '+str(a1[:5,:5]))
    dz1=np.multiply(np.transpose(self.weights_2).dot(dz2),np.maximum(0,np.sign(a1)))
    print('dz1-'+str(dz1[:5,:5]))
    dw1=(1/m)*dz1.dot(np.transpose(x_train))
    print('dw1-'+str(dw1[:5,:5]))
    db1=(1/m)*np.sum(dz1,axis=1,keepdims=True)
    print('db1- '+str(db1[:5,:5]))

    return dw1,db1,dw2,db2,dw3,db3
  
  def fit(self,x_train,y_train):
    num_classes=y_train.shape[0]
    num_features=x_train.shape[0]

    h_layer_1=self.h_layer_1
    h_layer_2=self.h_layer_2
    #initailizing weights(random)
    self.weights_1=np.random.rand(h_layer_1,num_features)*0.01
    self.weights_2=np.random.rand(h_layer_2,h_layer_1)*0.01
    self.weights_3=np.random.rand(num_classes,h_layer_2)*0.01
    self.dw1_cache=np.zeros((h_layer_1,num_features))
    self.dw2_cache=np.zeros((h_layer_2,h_layer_1))
    self.dw3_cache=np.zeros((num_classes,h_layer_2))
    self.bias_1=np.ones((h_layer_1,1))
    self.bias_2=np.ones((h_layer_2,1))
    self.bias_3=np.ones((num_classes,1))
    self.db1_cache=np.zeros((h_layer_1,1))
    self.db2_cache=np.zeros((h_layer_2,1))
    self.db3_cache=np.zeros((num_classes,1))

    #print(self.weights_1.shape)
    #print(self.weights_2.shape)
    #print(self.weights_3.shape)
    #print(self.bias_1.shape)
    #print(self.bias_2.shape)
    #print(self.bias_3.shape)

    error=[]
    accuracy=[]
    print('Initial weights- ' +str(self.weights_1[0,:5]))
    for i in range(self.epochs):
      print('Before FP')
      a_s=self.forward_prop(x_train)
      print('After FP')
      
      
      dw1,db1,dw2,db2,dw3,db3=self.back_prop(x_train,y_train,a_s[0],a_s[1],a_s[2])
      print('After BP')
      print("dw1- "+str(dw1[0:5,0:5]))
      print('dw2- '+str(dw2[0:5,0:5]))
      print('dw3- '+str(dw3[0:5,0:5]))

      self.optimize(dw1,db1,dw2,db2,dw3,db3)
      print('updated w1- ' +str(self.weights_1[0:5,:5]))
      print('updated w2- '+str(self.weights_2[0:5,:5]))
      print('updated w3- '+str(self.weights_3[0:5,:5]))
      epoch_error=self.calculate_error(a_s[-1],y_train)
      error.append(epoch_error)
      print("Epoch Error: "+str(i)+" "+str(epoch_error))
      print('After Error calculation')
      accuracy.append(self.accuracy(a_s[-1],y_train))
      print('After accuracy calculation')

    print(accuracy)  
    print(error)
  
  def accuracy(self,y_pred,y_actual):
    y_pred_T=np.transpose(y_pred)
    y_actual_T=np.transpose(y_actual)
    accuracy=accuracy_score(np.argmax(y_pred_T, axis=1), np.argmax(y_actual_T, axis=1))
    print("Accuracy: "+str(accuracy))
    return accuracy

  def calculate_error(self,y_pred,y_actual):
    #print(y_pred.shape)
    #print(y_actual.shape)
    print('Predicted- '+str(y_pred[:,0]))
    #print('Actual- '+str(y_actual[:,0]))
    y_pred=np.transpose(y_pred)
    y_actual=np.transpose(y_actual)
    return (-np.sum(y_actual * np.log(y_pred)))/y_pred.shape[0]
  def optimize_GD(self,dw1,db1,dw2,db2,dw3,db3):
    
    self.weights_1-=(self.learning_rate)*dw1
    self.weights_2-=(self.learning_rate)*dw2
    self.weights_3-=(self.learning_rate)*dw3
    self.bias_1-=(self.learning_rate)*db1
    self.bias_2-=(self.learning_rate)*db2
    self.bias_3-=(self.learning_rate)*db3


  def optimize(self,dw1,db1,dw2,db2,dw3,db3):
    rho=0.9  #user input, model parameter
    eps=10^-8
    
    #Updating caches
    self.dw1_cache=rho*self.dw1_cache+(1-rho)*np.multiply(dw1,dw1)
    self.dw2_cache=rho*self.dw2_cache+(1-rho)*np.multiply(dw2,dw2)
    self.dw3_cache=rho*self.dw3_cache+(1-rho)*np.multiply(dw3,dw3)

    self.db1_cache=rho*self.db1_cache+(1-rho)*np.multiply(db1,db1)
    self.db2_cache=rho*self.db2_cache+(1-rho)*np.multiply(db2,db2)
    self.db3_cache=rho*self.db3_cache+(1-rho)*np.multiply(db3,db3)

    #Gradient Descent

    self.weights_1=self.weights_1-(self.learning_rate/np.sqrt(self.dw1_cache))*dw1
    self.weights_2=self.weights_2-(self.learning_rate/np.sqrt(self.dw2_cache))*dw2
    self.weights_3=self.weights_3-(self.learning_rate/np.sqrt(self.dw3_cache))*dw3
    
    self.bias_1=self.bias_1-(self.learning_rate/np.sqrt(self.db1_cache))*db1
    self.bias_2=self.bias_2-(self.learning_rate/np.sqrt(self.db2_cache))*db2
    self.bias_3=self.bias_3-(self.learning_rate/np.sqrt(self.db3_cache))*db3    


  
  def evaluate(self,x_test,y_test):
    self.keep_prob_1=1
    self.keep_prob_2=1
    output=self.forward_prop(x_test)
    y_pred=output[2]
    y_pred_T=np.transpose(y_pred)
    y_actual_T=np.transpose(y_test)
    test_accuracy=accuracy_score(np.argmax(y_pred_T, axis=1), np.argmax(y_actual_T, axis=1))

    return test_accuracy

Problem #1.2 (10 points): Train your fully-connected neural network on the Fashion-MNIST dataset using 5-fold cross validation. Report accuracy on the folds, as well as on the test set.

In [4]:
# To simplify the usage of our dataset, we will be importing it from the Keras 
# library. Keras can be installed using pip: python -m pip install keras

# Original source for the dataset:
# https://github.com/zalandoresearch/fashion-mnist

# Reference to the Fashion-MNIST's Keras function: 
# https://keras.io/datasets/#fashion-mnist-database-of-fashion-articles

from keras.datasets import fashion_mnist

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


model=NeuralNetwork(100,0.01)
model.fit(np.transpose(x_train),np.transpose(y_train))

accuracy=model.evaluate(np.transpose(x_test),np.transpose(y_test))
print("Test accuracy: "+str(accuracy))

60000 train samples
10000 test samples
Initial weights- [0.00111582 0.0027048  0.0024879  0.00944885 0.00170055]
Before FP
After FP
a3-[[0.07298671 0.07143312 0.08199139 0.07859355 0.07583616]
 [0.09281446 0.09206868 0.09639817 0.09517741 0.09407331]
 [0.09379792 0.09308356 0.0970857  0.09597663 0.09495619]
 [0.06335265 0.06152068 0.07446197 0.07017003 0.06677032]
 [0.08808361 0.08712681 0.09302292 0.09127238 0.08973661]]
y_train-[[0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]]
dz3-[[ 0.07298671 -0.92856688 -0.91800861  0.07859355 -0.92416384]
 [ 0.09281446  0.09206868  0.09639817  0.09517741  0.09407331]
 [ 0.09379792  0.09308356  0.0970857   0.09597663  0.09495619]
 [ 0.06335265  0.06152068  0.07446197 -0.92982997  0.06677032]
 [ 0.08808361  0.08712681  0.09302292  0.09127238  0.08973661]]
dw3-[[-0.19558456 -0.19666691 -0.20017593 -0.1940934  -0.19463247]
 [ 0.0232976   0.02356961  0.02412443  0.02311367  0.02330467]
 [-0.12430109 -0.12499042

In [27]:
#k-fold cross validation
from keras.datasets import fashion_mnist

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

kf = KFold(n_splits = 5, shuffle = True)
validation_model=NeuralNetwork(50,0.01)
scores=[]
for i in range(5):
  result=next(kf.split(x_train),None)
  x_train_v=x_train[result[0]]
  x_test_v=x_train[result[1]]
  y_train_v=y_train[result[0]]
  y_test_v=y_train[result[1]]
  validation_model.fit(np.transpose(x_train_v),np.transpose(y_train_v))
  scores.append(validation_model.evaluate(np.transpose(x_test_v),np.transpose(y_test_v)))

print('Validation Accuracy: '+str(np.mean(scores)))
accuracy=validation_model.evaluate(np.transpose(x_test),np.transpose(y_test))
print("Test accuracy: "+str(accuracy))

60000 train samples
10000 test samples
Initial weights- [0.00892179 0.00025501 0.00153559 0.00392758 0.00781366]
Before FP
After FP
a3-[[0.08146329 0.08902693 0.08629647 0.08396925 0.08012152]
 [0.19413893 0.1601686  0.17270465 0.18314944 0.20003008]
 [0.03796421 0.05310226 0.04687439 0.04227495 0.03584163]
 [0.17123635 0.14721103 0.15628803 0.16365924 0.17523586]
 [0.06769402 0.07848436 0.07438835 0.07106803 0.06593431]]
y_train-[[0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]]
dz3-[[ 0.08146329 -0.91097307  0.08629647 -0.91603075  0.08012152]
 [ 0.19413893  0.1601686   0.17270465  0.18314944  0.20003008]
 [ 0.03796421  0.05310226  0.04687439  0.04227495 -0.96415837]
 [ 0.17123635  0.14721103 -0.84371197  0.16365924  0.17523586]
 [ 0.06769402  0.07848436  0.07438835  0.07106803  0.06593431]]
dw3-[[-0.14693911 -0.1471249  -0.1508333  -0.14694708 -0.14581634]
 [ 0.58566027  0.58673527  0.60076642  0.58556163  0.5821217 ]
 [-0.44560045 -0.44631871

KeyboardInterrupt: ignored

Problem #2.1 (40 points): Implement a Convolutional Neural Network from scratch. Similarly to problem 1.1, we will be implementing the same architecture as the one shown in [Keras' CNN documentation](https://keras.io/examples/mnist_cnn/). That is:

- Input layer
- Convolutional hidden layer with 32 neurons, a kernel size of (3,3), and relu activation function
- Convolutional hidden layer with 64 neurons, a kernel size of (3,3), and relu activation function
- Maxpooling with a pool size of (2,2)
- Dropout with a value of 0.25
- Flatten layer
- Dense hidden layer, with 128 neurons, and relu activation function
- Dropout with a value of 0.5
- Output layer, using softmax as the activation function

Our loss function is categorical crossentropy and the evaluation will be done using accuracy, as in Problem 1.1. However, we will not be using the gradient optimizer known as Adadelta.

In [0]:
class ConvolutionalNeuralNetwork(object):
  def __init__(epochs, learning_rate):
    pass
  
  def fit(self):
    pass
  
  def evaluate(self):
    pass

Problem #2.2 (10 points): Train your convolutional neural network on the Fashion-MNIST dataset using 5-fold cross validation. Report accuracy on the folds, as well as on the test set.

In [0]:
import keras
from keras.datasets import fashion_mnist

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)